In [1]:
from core import *
from tools import *
import numpy as np
import os
import tensorflow as tf
import keras as K

import sys
import argparse

Using TensorFlow backend.


In [2]:
# ICML_2019/DNA_CNN/Denoising

class ICE_Process:
    def __init__(self, n, k, nb_x_classes, nb_z_classes, x, z, param_name = 'test'):
        self.n, self.k, self.x, self.z, self.nb_x_classes, self.nb_z_classes = n, k, x, z, nb_x_classes, nb_z_classes
        self.param_name = param_name
        self.raw_error = error_rate(x,z)
        self.C = make_batch(z,k)
    
    def Approximate_E_step(self, pred_prob): # approximate E-step & M-step
        n, k, z, nb_x_classes, nb_z_classes = self.n, self.k, self.z, self.nb_x_classes, self.nb_z_classes
        
        """
        gamma[t][j] = p(x_t = j|Z_t,C_t;w)
        """
        
        # approximate E-step
        
        gamma = np.zeros((n, nb_x_classes))
        for i in range(nb_x_classes):
            gamma[:,i] = pred_prob[:,i+1]
        gamma[np.arange(n), z] += pred_prob[np.arange(n), 0]
        return gamma
        
    def M_step(self, pred_prob):
        n, k, z, nb_x_classes, nb_z_classes = self.n, self.k, self.z, self.nb_x_classes, self.nb_z_classes
        
        gamma = self.Approximate_E_step(pred_prob)
        
        # M-step
        PI = np.zeros((nb_x_classes, nb_z_classes))
        np.add.at(PI.T, z, gamma)
        PI /= (np.sum(gamma, axis = 0).reshape(nb_x_classes,1) + 1e-35)
        return PI
    
    def ICE(self, PI, PI_true): # Iterative Channel Estimation Process
        n, k, nb_x_classes, nb_z_classes, z, param_name, C = self.n, self.k, self.nb_x_classes, self.nb_z_classes, self.z, self.param_name, self.C
        iteration = 10
        
        train_loss, val_loss = [], []
        
        for t in range(iteration):
            # reset the L_new matrix
            L_new = L_NEW(PI, nb_x_classes, nb_z_classes)
            Y = make_pseudo_label(C, k, L_new)
            
            L_new_true = L_NEW(PI_true, nb_x_classes, nb_z_classes)
            Y_true = make_pseudo_label(C, k, L_new_true)
            
            model = NDUDE_CNN_model_5map(1000, nb_x_classes, nb_z_classes, k)
            
            # from second iteration, load previous weights and reset the learning rate.
            if t!=0:
                model.load_weights("weights/iteration/"+param_name+"_%d.hd5"%(t-1))
            
            # model training...
            hist = model.fit(C,Y,epochs=20, batch_size=2*4, verbose=1, validation_data=(C, Y_true))
            pred_prob = model.predict(C, batch_size = 20*4, verbose = 0)
            
            train_loss.append(hist.history['loss'][-1])
            val_loss.append(hist.history['val_loss'][-1])
            
            
            # resize the output
            N,D,_ = pred_prob.shape
            pred_prob = np.resize(pred_prob, (N*D,5))[:n]
            
            # estimate the channel
            PI = self.M_step(pred_prob)
            
            # save weights for next iteration
            model.save_weights("weights/iteration/"+param_name+"_%d.hd5"%(t))
            
            # save weights for denoising process
            if t == iteration-1:
                model.save_weights("weights/"+param_name+".hd5")
                save_PI(PI, param_name)
        return PI, train_loss, val_loss


In [3]:
PI_true = array([ [ 0.8122,  0.0034,  0.0894,  0.0950],
                  [ 0.0096,  0.8237,  0.0808,  0.0859],
                  [ 0.1066,  0.0436,  0.7774,  0.0724],
                  [ 0.0704,  0.0690,  0.0889,  0.7717]])
x, z = load_DNA(PI_true)

try:
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--k", help="window size k", type=int)
    parser.add_argument("--d", help="Assumed delta", type=float)
    
    args = parser.parse_args()
    
    result_name = sys.argv[0]
    k = args.k
    assumed_delta = args.d
    n = len(x)
    param_name = "ICE_%03d_%.2f"%(k, assumed_delta)
        
except:
    result_name = "test"
    k = 150
    assumed_delta = 0.40
    n = int(1e4)
    n = len(x)
    param_name = "test_%03d_%.2f"%(k, assumed_delta)
    
nb_x_classes, nb_z_classes = 4, 4
assumed_PI = sym_mat(4, assumed_delta)
x, z = x[:n], z[:n]

usage: ipykernel_launcher.py [-h] [--k K] [--d D]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1002/jupyter/kernel-96920016-7bea-4ed9-a1aa-d4250d6a6dbd.json


In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0, 1, 2, 3'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.backend.set_session(session)

In [5]:
f = open('results/'+result_name,'a')
print("k: %d assumed_delta: %.2f"%(k,assumed_delta))

ICE_N_DUDE = ICE_Process(n, k, nb_x_classes, nb_z_classes, x, z, param_name = param_name)
Estimated_PI, train_loss, val_loss = ICE_N_DUDE.ICE(assumed_PI, PI_true)
print(Estimated_PI)

k: 150 assumed_delta: 0.40
Train on 2470 samples, validate on 2470 samples
Epoch 1/20
2470/2470 [==============================] - 9s 4ms/step - loss: 0.8793 - val_loss: 0.7494
Epoch 2/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7953 - val_loss: 0.7345
Epoch 3/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7693 - val_loss: 0.7287
Epoch 4/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7557 - val_loss: 0.7250
Epoch 5/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7470 - val_loss: 0.7195
Epoch 6/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7411 - val_loss: 0.7199
Epoch 7/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7370 - val_loss: 0.7193
Epoch 8/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7337 - val_loss: 0.7168
Epoch 9/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7309 - val_loss: 0.7165
Epoch 1

2470/2470 [==============================] - 4s 2ms/step - loss: 0.7554 - val_loss: 0.6758
Epoch 19/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7553 - val_loss: 0.6758
Epoch 20/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7553 - val_loss: 0.6761
Train on 2470 samples, validate on 2470 samples
Epoch 1/20
2470/2470 [==============================] - 6s 2ms/step - loss: 0.7400 - val_loss: 0.6779
Epoch 2/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7396 - val_loss: 0.6771
Epoch 3/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7396 - val_loss: 0.6777
Epoch 4/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7396 - val_loss: 0.6769
Epoch 5/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7395 - val_loss: 0.6760
Epoch 6/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7395 - val_loss: 0.6772
Epoch 7/20
2470/2470 [=====================

2470/2470 [==============================] - 4s 2ms/step - loss: 0.7114 - val_loss: 0.6732
Epoch 17/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7113 - val_loss: 0.6739
Epoch 18/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7113 - val_loss: 0.6737
Epoch 19/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7113 - val_loss: 0.6742
Epoch 20/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7113 - val_loss: 0.6733
Train on 2470 samples, validate on 2470 samples
Epoch 1/20
2470/2470 [==============================] - 6s 3ms/step - loss: 0.7026 - val_loss: 0.6736
Epoch 2/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7025 - val_loss: 0.6747
Epoch 3/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7024 - val_loss: 0.6736
Epoch 4/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7024 - val_loss: 0.6725
Epoch 5/20
2470/2470 [===================

In [6]:
print(train_loss)

[0.718547298309774, 0.774111480365398, 0.7396645883799564, 0.7552730852293099, 0.7386023488121959, 0.7469703310896993, 0.7246565194747708, 0.7112910621561985, 0.7019372434751225, 0.7182151507269515]


In [8]:
print(val_loss)

[0.7116511434195978, 0.6772808345223246, 0.6788141081207677, 0.6760887978047977, 0.6766242048518378, 0.674336964060903, 0.6750607492470065, 0.6732962322138581, 0.6734274814003393, 0.6724951380660177]
